### <center id="p1"><h1> ⛓ `CHAINS` 🔗 - разбираемся с базовым элементом фрэймворка!</h1></center>

### Оглавление ноутбука
<img src='https://i.ytimg.com/vi/W3AoeMrg27o/maxresdefault.jpg' align="right" width="500" height="550" style="border-radius: 0.75rem;" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p1">🏃‍♀️‍➡️ Runnables - основные звенья цепи </a></li>
<li><a href="#plcel">LCEL - LangChain Expression Language 🤪🀄️</a></li>
<li><a href="#p4">🤖 TransformChain  </a></li>
<li><a href="#p5">🔗➕⛓ Вложенные цепочки</a></li>
<li><a href="#p6">Router Chain 🈯️ ➡️ 🇯🇵</a></li>
<li><a href="#p8">🧸 Выводы и заключения ✅ </a></li>
    
</ul></font></p>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 Как вы могли догадаться из названия, **цепочки (Chains)** - это один из фундаментальных строительных блоков фрэймворка `LangChain`. Они представляют собой просто цепочку компонентов, которые будут выполнены в определённом порядке. <br>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**Официальное определение цепочек:** <br>
Цепочка состоит из звеньев, которые могут быть как примитивами, так и другими цепочками. Примитивами могут быть либо промпты, LLM, utils, либо другие цепочки. Таким образом, цепочка — это, по сути, конвейер, который обрабатывает входные данные, используя определенную комбинацию примитивов. 

# <center id="p1"> 🏃‍♀️‍➡️ Runnables - основные звенья цепи </center> 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

📖 `Runnable` - наиболее близкий по смыслу перевод "запускаемый объект".

`Runnable` в `LangChain` — это базовый интерфейс, который лежит в основе работы с компонентами **LangChain**, такими как языковые модели, парсеры вывода, промпты и др. Он обеспечивает единообразный и предсказуемый способ взаимодействия с этими компонентами.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Основные характеристики `Runnable`:

* Позволяет вызывать компонент с одним входом и получать выход (`invoke`).
* Поддерживает пакетную обработку нескольких входов параллельно (`batch` и `batch_as_completed`), что улучшает производительность при обработке множества независимых запросов.
* Поддерживает потоковую выдачу результатов по мере их генерации (`stream`).
* Позволяет композировать несколько `Runnable`-компонентов вместе с помощью `LangChain Expression Language (LCEL)` для создания сложных конвейеров обработки - **цепочек**.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Простыми словами это любой объект из `LangChain`, который можно запустить/вызвать методом `invoke`.
В прошлых уроках мы уже создавали `Runnables`, вызывая llm и шаблон промпта, просто ещё не знали об этом.

In [5]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [6]:
!pip install langchain langchain-openai openai numexpr -q -U

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [7]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите ваш  API-ключ курса')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш  API-ключ курса ········


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Рассмотрим пример из прошлого урока: `ChatPromptTemplate` - это `Runnable`, как и модель `ChatOpenAI`.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Ты полезный AI-ассистент с чувством юмора."),
    ("user", "Расскажи мне шутку на тему - {topic}"),
])

prompt_template.invoke({"topic": "космос"})

ChatPromptValue(messages=[SystemMessage(content='Ты полезный AI-ассистент с чувством юмора.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне шутку на тему - космос', additional_kwargs={}, response_metadata={})])

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
И вот из таких `Runnables` и других примитивов, удобно собирать цепочки с помощью `LCEL`.

# <center id="plcel"> **`LCEL` - LangChain Expression Language** 🤪🀄️

`LCEL` `(LangChain Expression Language)` — это декларативный язык выражений в `LangChain`, предназначенный для построения и оркестровки цепочек (chains) из существующих компонентов `Runnable`. Вместо того чтобы описывать, как именно выполнять шаги, вы описываете, что должно произойти, а `LangChain` оптимизирует выполнение на этапе запуска.


Что он позволяет делать?:
* декларативный способ составлять цепочки
* стандартизирует интерфейсы
* проще персонализировать различные части цепочки
* позволяет легко заменять компоненты
* поддержка потоковой, пакетной и асинхронной обработки «из коробки»

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Звенья цепи записываются последовательно через символ **`|`**. Сама цепочка тоже становится `Runnable`.

</div>
Давайте соберём первую цепочку, которая будет менять стилистику текста:

In [10]:
# создадим шаблон и промпт
from langchain.prompts import PromptTemplate

template = '''Перепиши этот текcт в заданном стиле: {input_text}
Стиль: {style}.
Результат:'''

prompt = PromptTemplate(input_variables=['input_text', 'style'], template=template)

style_changer_chain = prompt | llm # И ВСЁ первая цепочка готова!

In [11]:
text = '''Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей, 
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.'''
style = 'Роман 18 века'

answer = style_changer_chain.invoke({'input_text': text, 'style': style})
print(answer)

content='Древствующие вас на курсе по преобразованию языковых моделей! Тут вас ожидает наставление по усовершенствованию моделей и раскрытию их потенциала в полной мере. Мы предоставим вам руководства о последних методах обучения и настройки языковых моделей, а также обучим вас практическим умениям для решения труднейших задач в обработке речи.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 185, 'total_tokens': 332, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bgfqghyu4cvY1m4XKOZiP7gJOPiDU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ba476a2b-a194-4ac4-a945-65ed44b85676-0' usage_metadata={'input_tokens': 185, 'output_tokens': 147, 'total_tokens': 

In [13]:
answer.content

'Древствующие вас на курсе по преобразованию языковых моделей! Тут вас ожидает наставление по усовершенствованию моделей и раскрытию их потенциала в полной мере. Мы предоставим вам руководства о последних методах обучения и настройки языковых моделей, а также обучим вас практическим умениям для решения труднейших задач в обработке речи.'

## 📖 Стандартный интерфейс всех `Runnables` имеет 3 метода:

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
 
 * `invoke` - обрабатывает одиночный запрос к модели
 * `batch` - обрабатывает список запросов к модели
 * `stream` - позволяет получать ответ модели в формате стриминга (можно увидеть,как модель генерирует ответ токен за токеном). Возвращает ответ модели в виде итерируемого объекта

Так же есть асинхронные реализации методов: `ainvoke`, `abatch`, `astream`. Для многопоточной работы.

In [17]:
style_changer_chain.batch([{'input_text': text, 'style': style},
                           {'input_text': text, 'style': 'поэма'}])

[AIMessage(content='Благословенствуйте, благородные слушатели на курсе мастерства в усовершенствовании языковых моделей! \nЗдесь мы преподаём умению достичь высшей совершенства моделей и раскрыть их бескрайний потенциал. \nМы предоставим вам советы, основанные на самых новейших методах обучения и настройки языковых моделей, \nи научим вас практическим умениям для решения сложнейших задач в деле обработки естественного языка.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 185, 'total_tokens': 373, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgfszBYX4hm8YZjY022ItQNEh50Bc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--875e98f3-14ea-4bea-a4c2-6a3bbd

In [18]:
# метод stream возвращает генератор
style_changer_chain.stream({'input_text': text, 'style': 'поэма'})

<generator object RunnableSequence.stream at 0x7f44e57804a0>

In [19]:
# Так можно просмотреть генерацию ответа токен за токеном - подходит для чат-ботов.
for s in style_changer_chain.stream({'input_text': text, 'style': 'поэма'}):
    print(s.content)


Д
а
 в
от
 и
 на
ст
ал
 д
ол
г
ож
д
анны
й
 м
ом
ент
,

К
ог
да
 пред
став
ляем
 в
ас
 сам
им
 с
еб
е
,

К
урс
 т
ю
н
ин
г
а
 я
з
ы
ков
ых
 мод
ел
ей
 –


И
с
ку
с
ств
о
 рас
к
р
ыв
ать
 и
х
 пот
ен
ци
ал
 до
 пред
ел
а
!


С
ов
ет
ы
 св
еж
ие
 мы
 в
ам
 д
ад
им
,

М
етод
ы
 об
уч
ения
 –
 нов
ей
ши
е
 найд
ем
.

На
строй
к
ой
 мод
ел
ей
 з
ай
м
ем
ся
 в
м
ест
е
,

И
 п
рак
ти
ч
ес
ки
е
 н
ав
ы
ки
 в
ам
 в
ру
ч
им
 –
 не
 в
ест
е
!


С
лож
ные
 зад
ач
и
 в
 об
работ
ке
 я
з
ы
ка


Р
аз
г
ад
аем
 в
м
ест
е
 с
 в
ами
,
 н
ав
ы
к
ами
 с
иль
ны
ми
.

П
у
сть
 слов
но
 по
э
ма
,
 к
 на
ш
ем
у
 к
урс
у
 ст
уп
ите
 –


И
 я
з
ы
ков
ые
 мод
ели
 д
ов
ед
ет
е
 до
 к
ру
т
ого
 п
ика
!


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**Добавим ещё звено:** Чтобы получать ответ от модели в виде строки, готовой для дальнейшего использования просто импортируем ещё один `Runnable` из `LangChain` - `StrOutputParser` и добавляем его в конец цепочки.


In [22]:
# Просто импортируем парсер и добавляем его, как ещё 1 звено цепочки
from langchain.schema.output_parser import StrOutputParser

chain_with_parser = prompt | llm | StrOutputParser() # И ВСЁ!

print(chain_with_parser.invoke({'input_text': text, 'style': style}))

Добро пожаловать на наш курс по усовершенствованию языковых моделей! Здесь мы учим, как сделать модели еще более совершенными и раскрыть их потенциал до предела. Мы предоставим вам сведения о самых актуальных методах обучения и настройки языковых моделей, а также научим вас практическим навыкам для решения сложнейших задач в обработке естественного языка.


<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Получили ответ от LLM, очищенный от доп. информации!

## <center id="p4"> 🤖 `TransformChain`

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

В качестве звеньев цепи могут выступать не только `Runnables`, но, например, и Python функции. 

Давайте представим, что мы получаем откуда-то сырой необработанный текст, который содержит пустые строки и много ненужных пробелов. Как вы помните, нам приходится считать количество израсходованных токенов, и не хочется их дополнительно тратить на ненужные лишние символы. К тому же такой текст выглядит неопрятно. 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Cоздадим цепочку, которая будет очищать поданный в неё текст от ненужных символов. Цепочки такого типа называют `Transform Chain`.

Для начала создадим функцию, которая будет убирать лишние пробелы и строки в исходном тексте.

In [23]:
import re

def del_spaces(inputs: dict) -> dict:
    text = inputs["input_text"]
    style = inputs["style"]
    
    # заменяем пустые строки и дополнительные пробелы на один, используя регулярные выражения
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"input_text": text, 'style': style}

In [24]:
# Возьмём наш предыдущий текст и "загрязним его"
dirty_text = '''Приветствуем на курсе по        тюнингу языковых        моделей! 

Тут мы учим, как сделать модельки еще круче и раскрыть     их потенциал до максимума. 
Мы дадим вам советы по   самым свежим методам  обучения и настройки языковых                    моделей,

а также научим практическим навыкам для        решения сложных задач в обработке естественного языка.'''

In [25]:
# Посмотрим на работу функции
print(del_spaces({'input_text': dirty_text, 'style': 'Рэп'})['input_text'])

Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей,
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Функция справляется со своей работой, и возвращает словарь в нужном формате!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
Теперь создадим цепочку, которая сначала будет чистить поданный в неё текст, а затем менять его стилистику.

In [26]:
seq_chain = del_spaces | prompt | llm |  StrOutputParser() # Готово!!!

print(seq_chain.invoke({'input_text': dirty_text, 'style': 'Рэп'}))

Yo, welcome to the language models tuning course!
Here we teach you how to make those models even tighter and unleash their full potential.
We'll give you advice on the freshest training and tuning methods,
and teach you practical skills to tackle tough tasks in natural language processing.


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Получили ответ!

## <center id="p5"> 🔗➕⛓ Вложенные цепочки

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
Рассмотрим пример с вложенными цепочками, когда необходимо одну цепочку использовать внутри другой.

In [28]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate


prompt1 = ChatPromptTemplate.from_template("В каком городе родился {person}?")
prompt2 = ChatPromptTemplate.from_template("В какой стране находится город {city}? Ответь на {language} языке.")

chain1 = prompt1 | llm | StrOutputParser()

# Ключ 'city' получаем из chain1, а ключ 'language' достаём из запроса с помощью itemgetter
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | llm
    | StrOutputParser()
)

result = chain2.invoke({"person": "Колумб", "language": "испанском"})
print(result)

El ciudad de Cristóbal Colón, marino e explorador italiano, nació en Génova en 1451, se encuentra en España.


## <center id="p6"> 🈯️ ➡️ 🇯🇵 Router Chain - определитель тем 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
<img src='../images/rout_chain.jpeg' align="right" width="600" height="600"  style="border-radius: 0.75rem;">   
    
📖  Если у вас есть две цепочки с LLM хорошо натренированными под разные задачи, одна хорошо разбирается в цветах, а другая в футболе. То, когда вы зададите вопрос, `Router Chain` может определить тему и отправить запрос в соответствующую цепочку. <br>

Рассмотрим на примере:

In [29]:
from typing import Literal
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from pydantic import BaseModel
from langchain_core.utils.function_calling import convert_to_openai_function

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
🫤 Так как у нас нет 2-х разных предобученых моделей под конкретную задачу, попросим ChatGPT отвечать нам, как-будто она специалист в 2 различных областях, с помощью промптов. 

In [30]:
botanist_template = """Ты очень опытный флорист и ботаник, знаешь всё о цветах, растениях.
Тебе нравится отвечать на вопросы о том, как выбирать и ухаживать за растениями. 
Ты отвечаешь так, что всё становится ясно даже начинающему цветоводу. 
Вот вопрос:
{input}"""

football_template = """Ты спортивный журналист с большим опытом, твоя основная специализация футбол.
Ты знаешь всё о футбольных командах и игроках, и очень любишь отвечать на вопросы о футболе, но кратко и по делу.
Вот вопрос:
{input}"""

In [31]:
# Создаём промпты
botanist_prompt = PromptTemplate.from_template(botanist_template)
football_prompt = PromptTemplate.from_template(football_template)

# Создаём ветки
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "botany", botanist_prompt),
    (lambda x: x["topic"] == "football", football_prompt),
    PromptTemplate.from_template("Answer the question: {input}"),
)

In [32]:
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"

    topic: Literal["botany", "football", "general"]
    "The topic of the user question. One of 'botany', 'football' or 'general'."


classifier_function = convert_to_openai_function(TopicClassifier)

model = ChatOpenAI(api_key=course_api_key, base_url="https://api.neuraldeep.tech/").bind(
    functions=[classifier_function],
    function_call={"name": "TopicClassifier"}
)
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = model | parser

In [33]:
# Посмотрим как работает цепочка-классификатор тем
classifier_chain.invoke("Какие цвета у Барселоны?")

'football'

In [34]:
classifier_chain.invoke("Сколько раз в неделю поливать кактус?")

'botany'

In [35]:
classifier_chain.invoke("Столица Хорватии?")

'general'

In [36]:
# Собираем цепочку-роутер
router_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch | llm | StrOutputParser()
)

<div class="alert alert-warning" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
А чтобы понять, как работает код выше или разобраться в других непонятных местах посмотрите отдельное [видео](https://stepik.org/lesson/1110883/step/6?unit=1122265).  Дадим [намёк](https://chat.langchain.com).

In [2]:
router_chain.invoke({'input': "Какие цвета у Барселоны?"})

Цвета Барселоны: синий и гранатовый


In [38]:
router_chain.invoke({'input':"Сколько раз в неделю поливать кактус?"})

'Обычно кактусу достаточно поливать раз в неделю в летний период и раз в две недели в зимний период. Однако это может зависеть от вида кактуса, его размера, условий выращивания и климата. Важно помнить, что переувлажнение кактусов может привести к гниению корней, поэтому лучше перебдеть и недополить, чем перелить растение. Всегда рекомендуется проверять влажность почвы перед поливом, чтобы найти оптимальный режим для конкретного экземпляра.'

In [39]:
router_chain.invoke({'input':"Столица Хорватии?"})

'Столица Хорватии - город Загреб.'

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 
✅ Получаем отличный результат!

# <center id="p8"> 🧸 Выводы и заключения ✅ <br>

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
* Во всех, приведенных выше, примерах мы могли обойтись каким-либо промптом.
* т.е. по сути можно было бы обходиться без цепочек, проектируя новый сложный промпт под каждую задачу. 
* С помощью цепочек `LangChain` это делать гораздо проще и удобнее. Особенно, если мы говорим про крупные сервисы. 
* Т.е. цепочки позволяют нам больше сосредоточиться на продумывании логики и функционала сервиса, а не на дизайне промптов.